In [ ]:
import os
import json
import time
import pytz
from datetime import datetime
import requests
import osmnx as ox
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import LineString
import networkx as nx
import hashlib
import json
import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm


In [ ]:
def generate_shape_key(shape):
    """Generate a unique hash key for a road shape based on its coordinates."""
    shape_str = json.dumps(shape, sort_keys=True)  # Convert to sorted JSON string
    return hashlib.md5(shape_str.encode()).hexdigest()  # Create hash

In [ ]:
PATH = "ISTANBUL"

GENERATION   = 300
POPULATION   = 200

AVERAGECHILD = 50
RANDOMCHILD  = 100
DIFFCHILD    = 150
BEST_CANDIDATE_OFFSPRING = 50
BEST_AROUND_WANDER = 50

SIMTIME      = -1

MAXCOSTVALUE = 9999999
constantFlow = 1
STEPSIZE     = 50
CHECKPOINT   = 20

highwayMap = {'motorway':0, 'highway':1, 'primary':2, 'secondary':3, 'tertiary':4, 'unclassified':5, 'residential':6, 'living_street': 7}

theta       =   [100, 50, 20, 17, 10, 8, 8, 5]
theta = np.array(theta)

In [ ]:
print("Loading for:" + PATH)
graph_path = os.path.join(PATH, "osmnx_graph.pkl")
roadKey_to_osmid_path = os.path.join(PATH, "roadKey_to_osmid.pkl")
realFlowData_path = os.path.join(PATH, "realFlowData")

with open(graph_path, 'rb') as f:
    G = pickle.load(f)

with open(roadKey_to_osmid_path, 'rb') as f:
    roadKeyToOsmIdMap = pickle.load(f)

print("Everything loaded correctly!")

In [ ]:
import os
import json
import numpy as np
import pickle
from tqdm import tqdm

with open(os.path.join("city_file_lists.pkl"), "rb") as f:
    city_file_lists = pickle.load(f)


# Initial jam factor coefficient
initialJamFactorCoefficient = 1

# Loop over cities and days
for city, city_data in city_file_lists.items():

    PATH = city
    print("Loading for:" + PATH)
    graph_path = os.path.join(PATH, "osmnx_graph.pkl")
    roadKey_to_osmid_path = os.path.join(PATH, "roadKey_to_osmid.pkl")
    realFlowData_path = os.path.join(PATH, "realFlowData")

    with open(graph_path, 'rb') as f:
        G = pickle.load(f)

    with open(roadKey_to_osmid_path, 'rb') as f:
        roadKeyToOsmIdMap = pickle.load(f)

    print("Everything loaded correctly!")


    for day, time_data in city_data["days"].items():
        #for time_section in ["morning", "night"]:
        for time_section in ["morning"]:
            
            realFlowData_path = os.path.join(city, "realFlowData")

            realFlowData = time_data[time_section]

            if not realFlowData:
                print(f"Skipping {city} - {day} - {time_section}: No data available.")
                continue

            print(f"\nProcessing: {city} - {day} - {time_section}")
            print("======\nPreparing data for training")

            # Initialize variables
            C = []
            osmIdToEdge = {}
            edgeToIndex = {}
            index = 0

            # Compute capacities and create mapping
            print("Starting loop over graph edges to calculate capacities and get maps.")
            for edge in tqdm(G.edges):
                
                highwayName = G.edges[edge]['highway']

                if type(highwayName) is list:
                    highwayName = highwayName[0] if highwayName else "residential"

                highwayName = highwayName.replace("_link", "")  
    
                if highwayName.startswith("trunk"):
                    highwayName = "primary"

                if highwayName.startswith("motorway"):
                    highwayName = "primary"

                if highwayName.startswith("highway"):
                    highwayName = "primary"
                
                if highwayName.startswith("living_street"):
                    highwayName = "residential"

                if highwayName not in highwayMap.keys():
                    highwayName = "residential"

                if highwayName.startswith("unclassified"):
                    highwayName = "residential"

                G.edges[edge]['highway'] = highwayName

                capacity = G.edges[edge]['length'] * theta[highwayMap[highwayName]]
                
                G.edges[edge]['capacity'] = capacity
                C.append(capacity) 

                osmId = G.edges[edge]['osmid']

                if type(osmId) is list:
                    for osmIdd in osmId:
                        osmIdToEdge[osmIdd] = edge
                else:
                    osmIdToEdge[osmId] = edge

                edgeToIndex[edge] = index
                index += 1

            print("C is generated. \nOsm2Edge and Edge2Index Maps are generated.")

            dataCount = len(realFlowData)
            edgeCount = len(G.edges)

            W = np.zeros((int(dataCount), int(edgeCount)))
            L = []

            t = 0
            dataAvailableEdges = set()
            print("\n======\nLooping around real flow data to gather jam factor information.")
            for dataFile in tqdm(realFlowData):
                with open(os.path.join(realFlowData_path, dataFile), "r") as f:
                    file = json.load(f)

                results = file['results']
                resultCount = 0

                avgJamFactor    = 0
                avgJamFactorW   = 0
                for data in results:
                    key = generate_shape_key(data['location']['shape'])

                    
                    try:
                        osmIds = roadKeyToOsmIdMap[key]
                        
                    except:
                        
                        continue
                    
                    if type(osmIds) is list:

                        for osmId in osmIds:
                            edge = osmIdToEdge.get(osmId)
                            if edge is None:
                                continue  # Skip if the edge isn't found

                            jamFactor = data['currentFlow']['jamFactor']

                            if (jamFactor > 10):
                                print("aaaaaa!!!!!!")
                                break
                        
                            load = (jamFactor / 10) * G.edges[edge]['capacity']

                            W[t, edgeToIndex[edge]] = load

                            avgJamFactorW += load / G.edges[edge]['capacity']

                            if t == 0:
                                dataAvailableEdges.add(edge)
                                avgJamFactor += jamFactor
                                resultCount += 1
                    else:
                        osmId = osmIds
                        edge = osmIdToEdge.get(osmId)
                        if edge is None:
                            continue  # Skip if the edge isn't found

                        jamFactor = data['currentFlow']['jamFactor']

                        if (jamFactor > 10):
                            print("aaaaaa!!!!!!")
                            break

                        load = (jamFactor / 10) * G.edges[edge]['capacity']

                        avgJamFactorW += load / G.edges[edge]['capacity']

                        W[t, edgeToIndex[edge]] = load

                        if t == 0:
                            dataAvailableEdges.add(edge)
                            resultCount += 1
                            avgJamFactor += jamFactor

                if t == 0:

                    avgJamFactor = (avgJamFactor / 10 / resultCount) * initialJamFactorCoefficient

                    for edge in G.edges:
                        if edge in dataAvailableEdges:
                            L.append(W[t, edgeToIndex[edge]])
                        else:
                            L.append(G.edges[edge]['capacity'] * avgJamFactor)

                t += 1

            # Collect available edge indices
            dataAvailableIndices = [edgeToIndex[edge] for edge in dataAvailableEdges]

            # Save training package for this city, day, and time section
            trainPackage = (G, C, L, W, dataAvailableIndices)

            if (np.sum(W/C > 1) > 0):
                print("ah")
                break
            
            # Define save path
            save_dir = os.path.join(city, time_section)
            os.makedirs(save_dir, exist_ok=True)  # Create subfolder if not exists
            save_path = os.path.join(save_dir, f"train_package_{day}.pkl")

            with open(save_path, "wb") as f:
                pickle.dump(trainPackage, f)

            print(f"Training package saved: {save_path}")
            print("Calculations are done.")